In [94]:
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

def extract_contour(color_image):
    hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 100, 100])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv_image, lower_green, upper_green)
    return mask

def calculate_border_irregularity(color_image):
    mask = extract_contour(color_image)
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    contours, _ = cv2.findContours(blurred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    largest_contour = max(contours, key=cv2.contourArea)
    perimeter = cv2.arcLength(largest_contour, True)
    area = cv2.contourArea(largest_contour)
    if area == 0:
        return None
    border_irregularity = perimeter ** 2 / (4 * np.pi * area)
    return border_irregularity

def calculate_color_variegation(image):
    lesion_r = image[:, :, 2]
    lesion_g = image[:, :, 1]
    lesion_b = image[:, :, 0]
    mean_r = np.mean(lesion_r)
    std_r = np.std(lesion_r)
    mean_g = np.mean(lesion_g)
    std_g = np.std(lesion_g)
    mean_b = np.mean(lesion_b)
    std_b = np.std(lesion_b)
    C_r = std_r / np.max(lesion_r) if np.max(lesion_r) != 0 else 0
    C_g = std_g / np.max(lesion_g) if np.max(lesion_g) != 0 else 0
    C_b = std_b / np.max(lesion_b) if np.max(lesion_b) != 0 else 0
    avg_color_variegation = (C_r + C_g + C_b) / 3
    return avg_color_variegation, mean_r, std_r, mean_g, std_g, mean_b, std_b

def calculate_diameter(image, dpi=1200):
    mask = extract_contour(image)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return 0
    max_contour = max(contours, key=cv2.contourArea)
    _, radius = cv2.minEnclosingCircle(max_contour)
    diameter_pixels = 2 * radius
    diameter_physical = diameter_pixels * (25.4 / dpi)
    return diameter_physical

def calculate_asymmetry(binary_image):
    binary_image = binary_image.astype(bool)
    horizontal_flip = np.fliplr(binary_image)
    diff_horizontal = binary_image & ~horizontal_flip
    vertical_flip = np.flipud(binary_image)
    diff_vertical = binary_image & ~vertical_flip
    diff_horizontal_area = np.count_nonzero(diff_horizontal)
    diff_vertical_area = np.count_nonzero(diff_vertical)
    area_total = np.count_nonzero(binary_image)
    if area_total == 0:
        return 0
    asymm_idx = ((diff_horizontal_area / area_total) + (diff_vertical_area / area_total)) / 2
    return asymm_idx

def predict_class(features, model_path, scaler_path):
    try:
        svm_model = joblib.load(model_path)
        scaler = joblib.load(scaler_path)
        features_scaled = scaler.transform([features])
        predicted_class = svm_model.predict(features_scaled)
        return predicted_class[0]
    except Exception as e:
        print(f"Error in predict_class: {str(e)}")
        return None

# Path gambar input
input_image = r"C:\Users\Danny\Skripsi\Dataset\data 2\restored_images_with_contours\mel\_56_3948599.jpg"

# Menghitung fitur-fitur dari gambar
image = cv2.imread(input_image)
if image is None:
    print(f"Error: Unable to read image '{input_image}'")
else:
    border_irregularity = calculate_border_irregularity(image)
    color_variegation, mean_r, std_r, mean_g, std_g, mean_b, std_b = calculate_color_variegation(image)
    diameter = calculate_diameter(image)
    binary_image = extract_contour(image)
    asymmetry = calculate_asymmetry(binary_image)

    # Ensure no feature is None
    if None in [border_irregularity, color_variegation, diameter, asymmetry]:
        print("Error: One or more features could not be calculated.")
    else:
        # Menampilkan nilai dari tiap fitur
        print(f"Border Irregularity: {border_irregularity}")
        print(f"Color Variegation: {color_variegation}")
        print(f"Mean R: {mean_r}, Std R: {std_r}")
        print(f"Mean G: {mean_g}, Std G: {std_g}")
        print(f"Mean B: {mean_b}, Std B: {std_b}")
        print(f"Diameter: {diameter}")
        print(f"Asymmetry: {asymmetry}")

        features = np.array([
            border_irregularity,
            color_variegation,
            mean_r,
            std_r,
            mean_g,
            std_g,
            mean_b,
            std_b,
            diameter,
            asymmetry
        ])

        # Path model SVM (file joblib)
        svm_model_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\svm.joblib"
        scaler_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\scaler.joblib"

        predicted_class = predict_class(features, svm_model_path, scaler_path)
        print(f"Predicted Class: {predicted_class}")


Border Irregularity: 2.6609176995254495
Color Variegation: 0.253918161691393
Mean R: 43.995196906887756, Std R: 74.40092288581273
Mean G: 39.75896843112245, Std G: 70.01274326030816
Mean B: 20.713608099489797, Std B: 35.28761894021951
Diameter: 5.021904708862305
Asymmetry: 0.8055054151624549
Predicted Class: 5


In [100]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pickle

# Path file CSV
file_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\coba\test.csv"

# Baca dataset dari file CSV
df = pd.read_csv(file_path)

# Mapping kelas ke nilai numerik
class_map = {'df': 0, 'nv': 1, 'mel': 2, 'vasc': 3, 'bkl': 5, 'akiec': 6}
df['Class'] = df['Class'].map(class_map)

# Pisahkan fitur (X) dan target (y)
X = df.drop('Class', axis=1).values
y = df['Class'].values

print(df.isnull().sum())  # Pemeriksaan nilai NaN di DataFrame

# Bagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Skalakan fitur menggunakan StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Simpan StandardScaler ke dalam file menggunakan pickle
scaler_file_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\scaler.pkl"
with open(scaler_file_path, 'wb') as f:
    pickle.dump(scaler, f)

print(f"Scaler saved to {scaler_file_path}")

# Inisialisasi model SVM dengan kernel RBF
model = SVC(kernel='rbf', C=10, gamma=1)

# Latih model menggunakan data latih
model.fit(X_train, y_train)

# Prediksi kelas untuk data uji
y_pred = model.predict(X_test)

# Evaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=class_map.keys()))

# Simpan model ke dalam file menggunakan pickle
model_file_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\svm.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved to {model_file_path}")


Class                  0
Asymmetry              0
Border Irregularity    0
Color Variegation      0
Diameter               0
Mean R                 0
Std R                  0
Mean G                 0
Std G                  0
Mean B                 0
Std B                  0
dtype: int64
Scaler saved to C:\Users\Danny\Skripsi\Dataset\data 2\file csv\scaler.pkl
Accuracy: 0.7463091482649842
Classification Report:
               precision    recall  f1-score   support

          df       0.77      0.79      0.78      2232
          nv       0.81      0.90      0.85      2477
         mel       0.65      0.66      0.65      2968
        vasc       0.87      0.90      0.88      2570
         bkl       0.64      0.56      0.59      2985
       akiec       0.76      0.74      0.75      2618

    accuracy                           0.75     15850
   macro avg       0.75      0.76      0.75     15850
weighted avg       0.74      0.75      0.74     15850

Model saved to C:\Users\Danny\Skripsi\Data

In [92]:
import pandas as pd
from sklearn.utils import resample

# Membaca data dari file CSV
file_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\coba\test.csv"
data = pd.read_csv(file_path)

# Menghapus duplikat data
data = data.drop_duplicates()

# Menampilkan distribusi kelas sebelum undersampling
print("Distribusi kelas sebelum undersampling:")
print(data['Class'].value_counts())

# Menentukan jumlah sampel untuk setiap kelas berdasarkan kelas dengan jumlah data terkecil
min_class_count = data['Class'].value_counts().min()

# Melakukan undersampling pada setiap kelas
undersampled_data_list = []
for class_value in data['Class'].unique():
    class_subset = data[data['Class'] == class_value]
    undersampled_class_subset = resample(class_subset,
                                         replace=False,    # mengambil sampel tanpa penggantian
                                         n_samples=min_class_count, # mengatur jumlah sampel sesuai dengan data minoritas
                                         random_state=42)  # untuk keperluan reproduktif
    undersampled_data_list.append(undersampled_class_subset)

# Menggabungkan kembali data yang telah di-undersampling
data_undersampled = pd.concat(undersampled_data_list)

# Menampilkan distribusi kelas setelah undersampling
print("Distribusi kelas setelah undersampling:")
print(data_undersampled['Class'].value_counts())

# Menyimpan data yang sudah di-undersampling ke file baru
output_file_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\coba\test_undersampled.csv"
data_undersampled.to_csv(output_file_path, index=False)

print(f"Data yang sudah di-undersampling telah disimpan ke '{output_file_path}'")


Distribusi kelas sebelum undersampling:
bkl      5920
mel      5920
vasc     5290
akiec    5209
nv       4949
df       4410
Name: Class, dtype: int64
Distribusi kelas setelah undersampling:
akiec    4410
bkl      4410
df       4410
mel      4410
nv       4410
vasc     4410
Name: Class, dtype: int64
Data yang sudah di-undersampling telah disimpan ke 'C:\Users\Danny\Skripsi\Dataset\data 2\file csv\coba\test_undersampled.csv'


In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib  # untuk versi scikit-learn <= 0.23
# from joblib import load, dump  # untuk versi scikit-learn >= 0.24

# Membaca dataset
df = pd.read_csv(r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\testku.csv")

# Memisahkan fitur dan target
X = df.drop('Class', axis=1)
y = df['Class']

# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skala fitur-fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Melatih model SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train_scaled, y_train)

# Prediksi menggunakan data uji
y_pred = svm_model.predict(X_test_scaled)

# Evaluasi model
print(classification_report(y_test, y_pred))

# Simpan model ke dalam file tanpa menggunakan pickle
joblib.dump(svm_model, 'svm_model.joblib')

# Memuat kembali model
loaded_model = joblib.load('svm_model.joblib')

# Contoh penggunaan model yang telah dimuat kembali
# Misalnya, prediksi untuk satu contoh baru
example_data = X_test.iloc[[0]]
example_data_scaled = scaler.transform(example_data)
predicted_class = loaded_model.predict(example_data_scaled)
print(f"Predicted class for example data: {predicted_class}")


              precision    recall  f1-score   support

       akiec       0.68      0.70      0.69      1057
          df       0.67      0.60      0.63       877
         mel       0.81      0.72      0.76      1183
          nv       0.82      0.97      0.89       971
        vasc       0.81      0.82      0.82      1068

    accuracy                           0.76      5156
   macro avg       0.76      0.76      0.76      5156
weighted avg       0.76      0.76      0.76      5156

Predicted class for example data: ['vasc']


In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib

# Membaca dataset
df = pd.read_csv(r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\test.csv")

# Memisahkan fitur dan target
X = df.drop('Class', axis=1)
y = df['Class']

# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Skala fitur-fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Melatih model SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train_scaled, y_train)

# Prediksi menggunakan data uji
y_pred = svm_model.predict(X_test_scaled)

# Evaluasi model
print(classification_report(y_test, y_pred))

# Simpan model ke dalam file tanpa menggunakan pickle
joblib.dump(svm_model, 'svm_model.joblib')

# Memuat kembali model
loaded_model = joblib.load('svm_model.joblib')

# Contoh penggunaan model yang telah dimuat kembali
# Misalnya, prediksi untuk satu contoh baru
example_data = X_test.iloc[[0]]
example_data_scaled = scaler.transform(example_data)
predicted_class = loaded_model.predict(example_data_scaled)
print(f"Predicted class for example data: {predicted_class}")


              precision    recall  f1-score   support

          df       0.80      0.87      0.83       859
         mel       0.88      0.73      0.80      1208
          nv       0.83      0.97      0.90       988
        vasc       0.84      0.82      0.83      1060

    accuracy                           0.84      4115
   macro avg       0.84      0.85      0.84      4115
weighted avg       0.84      0.84      0.84      4115

Predicted class for example data: ['vasc']


In [118]:
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

def extract_contour(color_image):
    hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 100, 100])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv_image, lower_green, upper_green)
    return mask

def calculate_border_irregularity(color_image):
    mask = extract_contour(color_image)
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    contours, _ = cv2.findContours(blurred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    largest_contour = max(contours, key=cv2.contourArea)
    perimeter = cv2.arcLength(largest_contour, True)
    area = cv2.contourArea(largest_contour)
    if area == 0:
        return None
    border_irregularity = perimeter ** 2 / (4 * np.pi * area)
    return border_irregularity

def calculate_color_variegation(image):
    lesion_r = image[:, :, 2]
    lesion_g = image[:, :, 1]
    lesion_b = image[:, :, 0]
    mean_r = np.mean(lesion_r)
    std_r = np.std(lesion_r)
    mean_g = np.mean(lesion_g)
    std_g = np.std(lesion_g)
    mean_b = np.mean(lesion_b)
    std_b = np.std(lesion_b)
    C_r = std_r / np.max(lesion_r) if np.max(lesion_r) != 0 else 0
    C_g = std_g / np.max(lesion_g) if np.max(lesion_g) != 0 else 0
    C_b = std_b / np.max(lesion_b) if np.max(lesion_b) != 0 else 0
    avg_color_variegation = (C_r + C_g + C_b) / 3
    return avg_color_variegation, mean_r, std_r, mean_g, std_g, mean_b, std_b

def calculate_diameter(image, dpi=1200):
    mask = extract_contour(image)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return 0
    max_contour = max(contours, key=cv2.contourArea)
    _, radius = cv2.minEnclosingCircle(max_contour)
    diameter_pixels = 2 * radius
    diameter_physical = diameter_pixels * (25.4 / dpi)
    return diameter_physical

def calculate_asymmetry(binary_image):
    binary_image = binary_image.astype(bool)
    horizontal_flip = np.fliplr(binary_image)
    diff_horizontal = binary_image & ~horizontal_flip
    vertical_flip = np.flipud(binary_image)
    diff_vertical = binary_image & ~vertical_flip
    diff_horizontal_area = np.count_nonzero(diff_horizontal)
    diff_vertical_area = np.count_nonzero(diff_vertical)
    area_total = np.count_nonzero(binary_image)
    if area_total == 0:
        return 0
    asymm_idx = ((diff_horizontal_area / area_total) + (diff_vertical_area / area_total)) / 2
    return asymm_idx

def predict_class(features, model_path, scaler_path):
    try:
        with open(model_path, 'rb') as f:
            svm_model = pickle.load(f)
        with open(scaler_path, 'rb') as f:
            scaler = pickle.load(f)
        
        features_scaled = scaler.transform([features])
        predicted_class = svm_model.predict(features_scaled)
        return predicted_class[0]
    except Exception as e:
        print(f"Error in predict_class: {str(e)}")
        return None

# Path gambar input
input_image = r"C:\Users\Danny\Skripsi\Dataset\data 2\restored_images_with_contours\vasc\_0_278127.jpg"

# Menghitung fitur-fitur dari gambar
image = cv2.imread(input_image)
if image is None:
    print(f"Error: Unable to read image '{input_image}'")
else:
    border_irregularity = calculate_border_irregularity(image)
    color_variegation, mean_r, std_r, mean_g, std_g, mean_b, std_b = calculate_color_variegation(image)
    diameter = calculate_diameter(image)
    binary_image = extract_contour(image)
    asymmetry = calculate_asymmetry(binary_image)

    # Ensure no feature is None
    if None in [border_irregularity, color_variegation, diameter, asymmetry]:
        print("Error: One or more features could not be calculated.")
    else:
        # Menampilkan nilai dari tiap fitur
        print(f"Border Irregularity: {border_irregularity}")
        print(f"Color Variegation: {color_variegation}")
        print(f"Mean R: {mean_r}, Std R: {std_r}")
        print(f"Mean G: {mean_g}, Std G: {std_g}")
        print(f"Mean B: {mean_b}, Std B: {std_b}")
        print(f"Diameter: {diameter}")
        print(f"Asymmetry: {asymmetry}")

        features = np.array([
            asymmetry,
            border_irregularity,
            color_variegation,
            diameter,
            mean_r,
            std_r,
            mean_g,
            std_g,
            mean_b,
            std_b,
        ])

        # Path model SVM (file pickle)
        svm_model_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\svm.pkl"
        scaler_path = r"C:\Users\Danny\Skripsi\Dataset\data 2\file csv\scaler.pkl"

        predicted_class = predict_class(features, svm_model_path, scaler_path)
        print(f"Predicted Class: {predicted_class}")


Border Irregularity: 1.5087183734739686
Color Variegation: 0.2955052219996837
Mean R: 41.17261240433673, Std R: 72.29918997866704
Mean G: 28.622229751275512, Std G: 58.69046734801325
Mean B: 27.443538743622447, Std B: 47.99713749977645
Diameter: 3.1974711736043293
Asymmetry: 0.8674179648657607
Predicted Class: 3


In [117]:
import cv2
import numpy as np

# Membuat array gambar
src = np.array([
    [6,  7,  8],
    [11, 0,  9],
    [13, 30, 22]
], dtype=np.uint8)

# Membuat mask biner
mask = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0]
], dtype=np.uint8)

# Melakukan inpainting dengan metode Telea
dst = cv2.inpaint(src, mask, 1, cv2.INPAINT_TELEA)

print("Gambar setelah inpainting:")
print(dst)


Gambar setelah inpainting:
[[ 6  7  8]
 [11 10  9]
 [13 30 22]]
